In [3]:
import pandas as pd

df = pd.read_csv('stock_min_volume_20250502full.csv')
df.head()

,REG_DATE,PAGE_TITLE,PAGE_TEXT
0,2025-05-01 00:00:02,NaN,나 어렸을때 네비가 없었는데 어떻게 운전한거여.. 맨날 가던길도 네비없으면 못가겠는...
1,2025-05-01 00:00:04,SKT 단독 청문회 확정,커뮤니티 2조 드립 삭제 => 국회 과방위 박제 위약금 면제 확답 회피 => 결정권...
2,2025-05-01 00:00:15,Trump Immigration,Trump Immigration A U.S. Marine Osprey is flow...
3,2025-05-01 00:00:15,Trump Immigration,Trump Immigration Migrants make their way to a...
4,2025-05-01 00:00:15,Trump Immigration,Trump Immigration Police face off with demonst...


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# 1. PDF 로드
loader = PyMuPDFLoader("dev_center_guide.pdf")
documents = loader.load()

print('pdf file load completed ..')

# 2. 텍스트 분할 (임베딩 효율을 위해)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

# 3. 임베딩 모델 정의
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. FAISS Vector 저장소 생성
vectorstore = FAISS.from_documents(splits, embedding)

# 5. 저장 (선택적)
vectorstore.save_local("model/dev_center_guide_index")

In [7]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # 또는 ChatOpenAI 등

# 1. Vector 저장소 로드
vectorstore = FAISS.load_local("model/dev_center_guide_index", embedding, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2. LLM 연결 (ChatGPT API Key 필요)
llm = OpenAI(temperature=0.1)

# 3. QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 4. 질의
query = "인앱 연동을 위한 전체 과정을 설명해줘."
result = qa_chain(query)

print("답변:\n", result["result"])

답변:
  
1. 원스토어 개발자센터에서 테스트 ID로 등록한 ID로 원스토어 로그인 후, 실결제를 한다.
2. 사용자가 문제를 해결하는데 도움이 되도록 딥링크를 제공한다.
3. 사용자가 문제를 해결하여 정기 결제가 복구되면 정기 결제일은 복구된 날짜로 변경된다.
4. 개발자가 자유롭게 구매취소를 할 수 있다.
5. 결제창 UI를 전면 또는 팝업 결제화면으로 선택할 수 있다.
6. 결제화면 설정은 원스토어 인앱결제 적용을 위한 사전준비 페이지의 'Android Manifest 파일 설정' 부분을 참고한다.
7. IAP SDK 또는 AIDL을 선택하여 연동할 수 있다.
8. 원스토어 서비스(ONE store service, OSS


In [20]:
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI  # 또는 ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from IPython.display import display, Markdown

# 🔹 1. 같은 임베딩 모델로 로딩
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🔹 2. 저장된 FAISS 인덱스 로드
vectorstore = FAISS.load_local("model/dev_center_guide_index", embedding, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 🔹 3. LLM 및 메모리 세팅
llm = OpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_tokens=2048
  )

memory = ConversationBufferMemory(
  memory_key="chat_history", 
  return_messages=True,
  output_key="answer"
  )

# 🔹 4. 대화형 QA 체인 생성
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer" 
)

# 🔹 5. 질의
query = "인앱 연동을 위한 전체 과정을 설명해줘."
response = qa_chain({"question": query})
# print("답변:\n", response["answer"])
display(Markdown(response["answer"]))  # 또는 파일로 저장

/home/ml/dev/python_env/jupyter_notebook/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 
1. 원스토어 개발자센터에서 테스트 ID로 등록한 ID로 원스토어 로그인
2. 실결제를 위해 개발자가 자유롭게 구매취소 가능
3. 개발자는 상용환경에서 실결제를 해볼 수 있음
4. 결제창 UI 설정 가능 (전면/팝업 결제화면 선택 가능)
5. 연동 방식 선택 가능 (IAP SDK, AIDL)
6. 원스토어 인앱결제 적용을 위한 사전준비 페이지의 'Android Manifest 파일 설정' 참고
7. 개발사의 검증 환경과 동일 수준의 원스토어 결제환경 제공
8. 개발사의 개발환경에서는 원스토어 Sandbox와 연동하여 개발/테스트 가능
9. 개발사의 상용환경에서는 원스토어 상용환경에서 실결제 가능

 
1. 원스토어 개발자센터에서 테스트 ID로 등록한 ID로 원스토어 로그인
2. 실결제를 위해 개발자가 자유롭게 구매취소 가능
3. 개발자는 상용환경에서 실결제를 해볼 수 있음
4. 결제창 UI 설정 가능 (전면/팝업 결제화면 선택 가능)
5. 연동 방식 선택 가능 (IAP SDK, AIDL)
6. 원스토어 인앱결제 적용을 위한 사전준비 페이지의 'Android Manifest 파일 설정' 참고
7. 개발사의 검증 환경과 동일 수준의 원스토어 결제환경 제공
8. 개발사의 개발환경에서는 원스토어 Sandbox와 연동하여 개발/테스트 가능
9. 개발사의 상용환경에서는 원스토어 상용환경에서 실결제 가능